# Fase 2: Limpieza y Transformación

#### 1. Importamos las librerías necesarias y agregamos la ruta de nuestro archivo .py

In [7]:
import sys
import pandas as pd

# Agregamos la ruta
sys.path.append('../')

# Importamos la función procesar_dataframe de limpieza.py
from src.limpieza import procesar_dataframe

# Importamos el diccionario de variables.py
from src.variables import cambiar_tipo_columnas

#### 2. Cargamos el dataframe transformado, hacemos llamada a la función y comprobamos los cambios

In [8]:
# Cargamos el DataFrame
df_csv = pd.read_csv('../data/transform_data/transform_Data.csv', low_memory=False)

# Verificamos los nombres de las columnas en el DataFrame
print("Columnas del DataFrame antes de procesar:")
print(df_csv.columns)

# Verificamos que el diccionario cambiar_tipo_columnas está bien definido
print("Diccionario cambiar_tipo_columnas:")
print(cambiar_tipo_columnas)

# Rellenamos valores nulos para evitar errores en la conversión
df_csv.fillna(value=0, inplace=True)

# Llamamos a la función procesar_dataframe con los diccionarios importados
df = procesar_dataframe(df_csv, cambiar_tipo_columnas=cambiar_tipo_columnas)

# Verificamos el resultado mostrando los tipos de datos después de la transformación
print("Tipos de datos después de procesar:")
print(df.dtypes)

Columnas del DataFrame antes de procesar:
Index(['club_id', 'club_code', 'name', 'domestic_competition_id',
       'total_market_value', 'squad_size', 'average_age', 'foreigners_number',
       'foreigners_percentage', 'national_team_players', 'stadium_name',
       'stadium_seats', 'net_transfer_record', 'coach_name', 'last_season',
       'filename', 'url', 'game_id', 'own_goals', 'own_position',
       'own_manager_name', 'opponent_id', 'opponent_goals',
       'opponent_position', 'opponent_manager_name', 'hosting', 'is_win'],
      dtype='object')
Diccionario cambiar_tipo_columnas:
{'net_transfer_record': 'float64', 'coach_name': 'object', 'own_position': 'int', 'opponent_position': 'int', 'opponent_id': 'int', 'opponent_goals': 'int', 'hosting': 'category'}
Tipos de datos después de procesar:
club_id                       int64
club_code                    object
name                         object
domestic_competition_id      object
total_market_value          float64
squad_size

#### 3. Analizamos si existen o no valores duplicados, para poder reducir el tamaño del archivo y mejorar la velocidad del fichero

In [9]:
# Comprobamos si existen valores duplicados
if df.duplicated().values.any():
    print("Existen valores duplicados por eliminar")
else:
    print("No existen valores duplicados")

# Comprobamos y verificamos que no existen valores duplicados
df.duplicated().sum()

No existen valores duplicados


np.int64(0)

#### 4. Analizamos si existen o no valores nulos

In [10]:
# Calculamos cuantos valores nulos existen
print("Los valores nulos existentes por columna son:")
df.isnull().sum()

Los valores nulos existentes por columna son:


club_id                         0
club_code                       0
name                            0
domestic_competition_id         0
total_market_value              0
squad_size                      0
average_age                     0
foreigners_number               0
foreigners_percentage           0
national_team_players           0
stadium_name                    0
stadium_seats                   0
net_transfer_record        120099
coach_name                      0
last_season                     0
filename                        0
url                             0
game_id                         0
own_goals                       0
own_position                    0
own_manager_name                0
opponent_id                     0
opponent_goals                  0
opponent_position               0
opponent_manager_name           0
hosting                         0
is_win                          0
dtype: int64

In [11]:
# Calculamos el % de nulos en cada columna, para saber la cantidad de nulos que hemos obtenido
df.isnull().mean() * 100

club_id                      0.0
club_code                    0.0
name                         0.0
domestic_competition_id      0.0
total_market_value           0.0
squad_size                   0.0
average_age                  0.0
foreigners_number            0.0
foreigners_percentage        0.0
national_team_players        0.0
stadium_name                 0.0
stadium_seats                0.0
net_transfer_record        100.0
coach_name                   0.0
last_season                  0.0
filename                     0.0
url                          0.0
game_id                      0.0
own_goals                    0.0
own_position                 0.0
own_manager_name             0.0
opponent_id                  0.0
opponent_goals               0.0
opponent_position            0.0
opponent_manager_name        0.0
hosting                      0.0
is_win                       0.0
dtype: float64

#### 5. Conclusiones del cálculo de % de valores nulos

**Calculamos el % de valores nulos en cada columna, y obtenemos las siguientes conclusiones:**

1. Si una columna contiene el 100% de valores nulos, lo más eficiente es eliminar dicha columna (total_market_value, coach_name)<br>

2. Si menos del 5% de los valores de la columna son nulos, podemos sustituirlos por la mediana en columnas numéricas y la moda en columnas categóricas.<br>

3. Sustituir por la moda es adecuado para las columnas categóricas, ya que la moda es el valor más frecuente.<br>

4. Sustituir por la mediana es útil cuando la distribución de los datos es asimétrica (sesgada), ya que la mediana no se ve afectada por los valores extremos.<br>

In [12]:
# Según el análisis eliminamos la columna net_transfer_record, puesto que tiene el 100% de valores nulos. Eliminamos además columnas que no serán de utilidad
df = df.drop(['net_transfer_record', 'total_market_value', 'coach_name', 'club_code'], axis=1)

In [13]:
# Sustituimos los nulos en las columnas numéricas, por la mediana de cada columna
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())

In [14]:
# Sustituimos los nulos en las columnas categóricas, por la moda de cada columna
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])

In [15]:
# Verificamos que ya no hay valores nulos
df.isnull().sum()

club_id                    0
name                       0
domestic_competition_id    0
squad_size                 0
average_age                0
foreigners_number          0
foreigners_percentage      0
national_team_players      0
stadium_name               0
stadium_seats              0
last_season                0
filename                   0
url                        0
game_id                    0
own_goals                  0
own_position               0
own_manager_name           0
opponent_id                0
opponent_goals             0
opponent_position          0
opponent_manager_name      0
hosting                    0
is_win                     0
dtype: int64

In [16]:
# Verificamos que se aplicaron los cambios
df.head()

,club_id,name,domestic_competition_id,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,...,game_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,105,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,...,2581153,2.0,10,Dirk Schuster,42,2,9,Michael Frontzeck,Home,0
1,105,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,...,2581674,2.0,10,Dirk Schuster,39,3,8,Martin Schmidt,Home,0
2,105,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,...,2704228,1.0,12,Norbert Meier,24,0,9,Niko Kovac,Home,1
3,105,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,...,4096140,0.0,18,Torsten Lieberknecht,15,2,1,Xabi Alonso,Home,0
4,105,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,...,2361070,1.0,0,Dirk Schuster,33,3,0,Jens Keller,Home,0


#### 6. Analizamos las columnas categóricas

In [17]:
# Verificamos posibles erratas o valores mal escritos, con head mostramos los 20 valores de mayor frecuencia
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n🔹 Columna: {col}")
    print(df_csv[col].value_counts(dropna=False).head(20))


🔹 Columna: name
name
Real Madrid Club de Fútbol                    709
Futbol Club Barcelona                         706
Sevilla Fútbol Club S.A.D.                    687
Club Atlético de Madrid S.A.D.                674
Manchester City Football Club                 670
Chelsea Football Club                         663
Manchester United Football Club               658
Juventus Football Club                        655
Liverpool Football Club                       643
Arsenal Football Club                         641
The Celtic Football Club                      636
FC Bayern München                             636
Tottenham Hotspur Football Club               630
Società Sportiva Calcio Napoli                624
Associazione Sportiva Roma                    622
Società Sportiva Lazio S.p.A.                 619
Borussia Dortmund                             612
Athletic Club Bilbao                          610
AFC Ajax Amsterdam                            609
Football Club Internazionale

In [18]:
# Eliminamos espacios en blanco, espacios dobles o caracteres
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].replace(r'\s+', ' ', regex=True)

#### 7. Analizamos las columnas numéricas

In [19]:
# Revisamos las estadísticas generales
df.select_dtypes(include=["number"]).describe()

,club_id,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_seats,last_season,game_id,own_goals,own_position,opponent_id,opponent_goals,opponent_position,is_win
count,120099.000000,120099.000000,120099.000000,120099.000000,120099.000000,120099.000000,120099.000000,120099.000000,1.200990e+05,120099.000000,120099.0,120099.0,120099.0,120099.0,120099.000000
mean,3203.647224,26.042740,24.670431,13.238961,48.326645,6.955936,30278.971215,2022.806759,3.107617e+06,1.448214,7.802396,3574.429396,1.378496,7.802396,0.396265
std,8386.558570,6.477525,5.423141,6.052661,21.297589,5.528892,19318.398260,2.508307,6.342387e+05,1.346801,5.97231,9112.032868,1.298949,5.97231,0.489123
min,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2012.000000,2.211607e+06,0.000000,0.0,1.0,0.0,0.0,0.000000
25%,294.000000,25.000000,24.600000,10.000000,36.000000,2.000000,14708.000000,2023.000000,2.578184e+06,0.000000,2.0,306.0,0.0,2.0,0.000000
50%,865.000000,27.000000,25.800000,14.000000,50.000000,6.000000,27084.000000,2024.000000,3.055072e+06,1.000000,7.0,903.0,1.0,7.0,0.000000
75%,2451.000000,29.000000,26.700000,17.000000,65.400000,11.000000,42115.000000,2024.000000,3.598382e+06,2.000000,13.0,2672.0,2.0,13.0,1.000000
max,110302.000000,41.000000,29.800000,29.000000,100.000000,22.000000,81365.000000,2024.000000,4.500281e+06,16.000000,21.0,121966.0,17.0,21.0,1.000000


In [20]:
# Detectamos valores fuera de rango, por ejemplo si obtenemos una edad negativa
for col in df.select_dtypes(include=['number']).columns:
    print(f"\n🔹 Columna: {col}")
    print(f"Valores únicos: {df[col].nunique()}")
    print(f"Máximo: {df[col].max()}, Mínimo: {df[col].min()}")


🔹 Columna: club_id
Valores únicos: 439
Máximo: 110302, Mínimo: 3

🔹 Columna: squad_size
Valores únicos: 32
Máximo: 41, Mínimo: 0

🔹 Columna: average_age
Valores únicos: 74
Máximo: 29.8, Mínimo: 0.0

🔹 Columna: foreigners_number
Valores únicos: 28
Máximo: 29, Mínimo: 0

🔹 Columna: foreigners_percentage
Valores únicos: 171
Máximo: 100.0, Mínimo: 0.0

🔹 Columna: national_team_players
Valores únicos: 22
Máximo: 22, Mínimo: 0

🔹 Columna: stadium_seats
Valores únicos: 401
Máximo: 81365, Mínimo: 0

🔹 Columna: last_season
Valores únicos: 13
Máximo: 2024, Mínimo: 2012

🔹 Columna: game_id
Valores únicos: 63372
Máximo: 4500281, Mínimo: 2211607

🔹 Columna: own_goals
Valores únicos: 17
Máximo: 16.0, Mínimo: 0.0

🔹 Columna: own_position
Valores únicos: 22
Máximo: 21, Mínimo: 0

🔹 Columna: opponent_id
Valores únicos: 1739
Máximo: 121966, Mínimo: 1

🔹 Columna: opponent_goals
Valores únicos: 17
Máximo: 17, Mínimo: 0

🔹 Columna: opponent_position
Valores únicos: 22
Máximo: 21, Mínimo: 0

🔹 Columna: is_

#### 8. Cálculo del Método de Rango Intercuartílico (IQR)

In [21]:
# Detectamos valores bajos o altos que podrían causar conflicto
import numpy as np

for col in df.select_dtypes(include=['number']).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    print(f"\n🔹 Columna: {col} - Outliers encontrados: {outliers.shape[0]}")


🔹 Columna: club_id - Outliers encontrados: 14629

🔹 Columna: squad_size - Outliers encontrados: 8043

🔹 Columna: average_age - Outliers encontrados: 6280

🔹 Columna: foreigners_number - Outliers encontrados: 671

🔹 Columna: foreigners_percentage - Outliers encontrados: 0

🔹 Columna: national_team_players - Outliers encontrados: 0

🔹 Columna: stadium_seats - Outliers encontrados: 0

🔹 Columna: last_season - Outliers encontrados: 20445

🔹 Columna: game_id - Outliers encontrados: 0

🔹 Columna: own_goals - Outliers encontrados: 1335

🔹 Columna: own_position - Outliers encontrados: 0

🔹 Columna: opponent_id - Outliers encontrados: 14831

🔹 Columna: opponent_goals - Outliers encontrados: 1068

🔹 Columna: opponent_position - Outliers encontrados: 0

🔹 Columna: is_win - Outliers encontrados: 0


#### 9. Guardamos el CSV con la limpieza final realizada

In [22]:
df.to_csv('../data/transform_data/data_clean.csv', index=False)